In [ ]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
from sklearn.preprocessing import StandardScaler

In [ ]:
video_folder = '/Users/andrei-macpro/Documents/Data/openpose/play/tracking/tracking/1080_play'
# do it for one folder first
# load each file in a pandas dataframe

tracks = []
for filename in sorted(os.listdir(video_folder)):
        # load the file in a pandas dataframe
    if filename == ".DS_Store":
        continue    
    print(filename)
    filepath = os.path.join(folder_name, filename)
    df = pd.read_csv(filepath, index_col=0)
    # Remove rows with duplicated index values because the interpolation sometimes created duplicates
    df = df[~df.index.duplicated(keep='first')]
    df = clean_outliers(df)
    tracks.append(df)


In [ ]:
cg = tracks[0]
child = tracks[1]

In [ ]:
# Check if cg has duplicate indices
if cg.index.duplicated().any():
    print("cg has duplicate indices")

# Check if child has duplicate indices
if child.index.duplicated().any():
    print("child has duplicate indices")

In [ ]:
duplicate_indices_child = child[child.index.duplicated(keep=False)]
print(duplicate_indices_child)

In [ ]:
combined = pd.concat([cg, child], axis=1)

In [ ]:
combined


In [ ]:
# select first column from df 
df.iloc[:, 0]

In [ ]:
# Calculate z-scores for x and y columns
# make this code into a function that takes a dataframe and returns the cleaned dataframe

from scipy.stats import zscore
def clean_outliers(df):
    z_scores_x = zscore(df.iloc[:,0])
    z_scores_y = zscore(df.iloc[:,1])

    # Define threshold for z-score (e.g., 3)
    threshold = 3

    # Identify outliers based on z-scores
    outliers_x = df[abs(z_scores_x) > threshold]
    outliers_y = df[abs(z_scores_y) > threshold]

    # Remove outliers from DataFrame
    cleaned_df = df[~df.index.isin(outliers_x.index) & ~df.index.isin(outliers_y.index)]
    return cleaned_df

In [ ]:
combined_scaled[len(child):]

In [ ]:

# Combine both datasets for joint scaling

scaler = StandardScaler()
combined_scaled = scaler.fit_transform(combined_data)

# Split the scaled data back into caregiver and child data
scaled_caregiver = combined_scaled[:len(cg)]
scaled_child = combined_scaled[len(cg):]

cg[['x', 'y']] = scaled_caregiver
child[['x', 'y']] = scaled_child




In [ ]:
cg

In [ ]:
aligned_positions = pd.merge(child, cg, on='frame', suffixes=('_caregiver', '_child'))
print("Aligned Positions Data:")
print(aligned_positions)

In [ ]:
aligned_positions.index

In [ ]:
distances = []
frame_indices = aligned_positions.index
for i in range(len(aligned_positions) - 1):
    if frame_indices[i + 1] - frame_indices[i] < 5:
        distance = np.sqrt((aligned_positions['x_caregiver'].iloc[i] - aligned_positions['x_child'].iloc[i])**2 + 
                           (aligned_positions['y_caregiver'].iloc[i] - aligned_positions['y_child'].iloc[i])**2)
        distances.append(distance)
    else:
        distances.append(np.nan)  # Or some placeholder for frames that don't meet the criteria

distances


In [ ]:
distances

In [ ]:
# ok but what if there are actually more gaps than just the one ? how do we keep both the aligned positions and the distances in sync?
distances

In [ ]:
# do it for all now 
from moviepy.editor import VideoFileClip
video_folder = '/Users/andrei-macpro/Documents/Data/videos/meal_videos' 

durations = []
file_names = []

for folder_name in sorted(os.listdir(video_folder)):
    if folder_name == ".DS_Store":
        continue
    file_path = os.path.join(video_folder, folder_name)
    if os.path.isfile(file_path):
        clip = VideoFileClip(file_path)
        durations.append(clip.duration)
        file_names.append(folder_name.split('.')[0])

durations = pd.DataFrame({'file_name': file_names, 'duration': durations})


In [ ]:
durations

In [ ]:
# how to select a row by its cell name in a dataframe
df.loc[df['file_name'] == '1043_play']['duration'].values[0]

In [ ]:
# save df to csv in the folder /Users/andrei-macpro/Documents/Data/videos/durations
df.to_csv('/Users/andrei-macpro/Documents/Data/videos/durations/durations_play.csv')

In [ ]:
tracking_folder = '/Users/andrei-macpro/Documents/Data/openpose/meal/tracking/meal_tracking_2' 

In [ ]:
# now do it for all videos
stats = {}
scaler = StandardScaler()
for folder_name in sorted(os.listdir(tracking_folder)):
    if folder_name == ".DS_Store":
        continue
    file_path = os.path.join(tracking_folder, folder_name)
    tracks = {}
    for file in sorted(os.listdir(file_path)):
                # load the file in a pandas dataframe
            if file == ".DS_Store":
                continue    
            filepath = os.path.join(tracking_folder,folder_name, file)
            df = pd.read_csv(filepath, index_col=0)
            df = df[~df.index.duplicated(keep='first')]
            df.columns = ['x_' + file.split('.')[0], 'y_' + file.split('.')[0]]
            df = clean_outliers(df)
            tracks[file.split('.')[0]] = df

    print(folder_name)
    combined_data = pd.concat([df for df in tracks.values()], axis=1)
    combined_scaled = scaler.fit_transform(combined_data)
    # Convert the scaled data back to a DataFrame
    combined_scaled_df = pd.DataFrame(combined_scaled, index=combined_data.index, columns=combined_data.columns)

# Split the combined_scaled_df back into separate DataFrames and assign them back to the tracks dictionary
    for track_name in tracks.keys():
        x_cols = [col for col in combined_scaled_df.columns if col.startswith('x_' + track_name)]
        y_cols = [col for col in combined_scaled_df.columns if col.startswith('y_' + track_name)]
        tracks[track_name] = combined_scaled_df[x_cols + y_cols]

    # Calculate Euclidean distances
    for track_name, track_df in tracks.items():
        distances = []
        frame_indices = track_df.index
        
        for i in range(len(track_df)-1):
            
            if frame_indices[i + 1] - frame_indices[i] < 5:
                distance = np.sqrt((track_df['x_' + track_name].iloc[i] - track_df['x_' + track_name].iloc[i+1])**2 + 
                            (track_df['y_' + track_name].iloc[i] - track_df['y_' + track_name].iloc[i+1])**2)
                distances.append(distance)
            else:
                distances.append(np.nan)
        
         # Append np.nan to match the number of rows in the DataFrame
        distances.append(np.nan)
        track_df['distance'] = distances

        duration = durations.loc[durations['file_name'] == folder_name]['duration'].values[0]
        mean_movement = track_df['distance'].mean() / duration*100
        median_movement = track_df['distance'].median() / duration*100
        var_movement = track_df['distance'].var() / duration*100
        std_movement = track_df['distance'].std() / duration*100
        min_movement = track_df['distance'].min() / duration*100
        max_movement =  track_df['distance'].max() / duration*100
        skewness_movement = track_df['distance'].skew() / duration*100
        kurtosis_movement = track_df['distance'].kurtosis() / duration*100

        
        if folder_name not in stats:
            stats[folder_name] = {}
        stats[folder_name][track_name] = {'movement_mean': mean_movement, 'movement_var': var_movement, 'movement_std': std_movement, 'movement_min': min_movement, 'movement_max': max_movement, 'movement_skew': skewness_movement, 'movement_kurt': kurtosis_movement}

# normalize the distances (mean, variance etc) by the duration of the video
    

In [ ]:
# Convert the stats dictionary to a DataFrame
df = pd.DataFrame.from_dict({(i,j): stats[i][j] 
                           for i in stats.keys() 
                           for j in stats[i].keys()},
                           orient='index')

# Set the index names
df.index.names = ['s_id', 'track_name']

In [ ]:
df

In [ ]:
# Reset only the second level of the index (track_name)
df.reset_index(level=1, inplace=True)

# Create new DataFrame to hold results
new_df = pd.DataFrame(index=df.index.unique())

# Loop over each unique track_name
for track_name in df['track_name'].unique():
    # Select rows for this track_name
    temp_df = df[df['track_name'] == track_name].copy()
    # Drop the 'track_name' column as it's no longer needed
    temp_df.drop(columns=['track_name'], inplace=True)
    # Add the track_name as a prefix to each column name
    temp_df.columns = [f'{track_name}_{col}' for col in temp_df.columns]
    # Add the results to new_df
    new_df = new_df.join(temp_df)

new_df

In [ ]:
# save df to /Users/andrei-macpro/Documents/Data/tracking/features
new_df.to_csv('/Users/andrei-macpro/Documents/Data/tracking/features/meal/movement_each.csv')